In [1]:
import re
def rm_tags(text):
    re_tag = re.compile(r'<[^>]+>')
    return re_tag.sub('', text) #用空字符替换

In [2]:

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, Dropout, Input, concatenate
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Model
import os
import tarfile
import numpy as np

Using TensorFlow backend.


In [3]:
import keras

In [189]:
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
# import tensorflow as tf

In [4]:
#构造函数读取train和test中的所有影评
def read_files(filetype):
    """
    filetype: 'train' or 'test'
    return type: 
                all_texts:filetype数据文本
                all_labels:filetype数据标签
    """
    all_labels = [1]*12500 + [0]*12500
    all_texts = []
    file_list = []
    path = r'./aclImdb/'  #r 表示./aclImdb/是raw string 不需要转义
    #读取pos文本名
    pos_path = path + filetype + '/pos/'
    for file in os.listdir(pos_path):
        file_list.append(pos_path+file)
    #读取neg文本名
    neg_path = path + filetype + '/neg/'
    for file in os.listdir(neg_path):
        file_list.append(neg_path+file)
    print(file_list[0])
    #将pos文本和neg文本添加到all_texts
    for file_name in file_list:
        with open(file_name, encoding = 'utf-8') as f:
            all_texts.append(rm_tags("".join(f.readlines())))
    return all_texts, all_labels

In [5]:
seq = ("a", "b", "c") # 字符串序列
print ("".join(seq))

abc


In [6]:
#将评论数据和标签分别载入
x_train,y_train  = read_files('train')

./aclImdb/train/pos/0_9.txt


In [106]:
x_train[0]

'Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!'

In [107]:
len(y_train)

25000

In [7]:
#用keras进行分词
for cut in range(len(x_train)):
     x_train[cut]= keras.preprocessing.text.text_to_word_sequence(x_train[cut],
                                               filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~\t\n',
                                               lower=True,
                                               split=" ")

In [8]:
x_train[0]

['bromwell',
 'high',
 'is',
 'a',
 'cartoon',
 'comedy',
 'it',
 'ran',
 'at',
 'the',
 'same',
 'time',
 'as',
 'some',
 'other',
 'programs',
 'about',
 'school',
 'life',
 'such',
 'as',
 'teachers',
 'my',
 '35',
 'years',
 'in',
 'the',
 'teaching',
 'profession',
 'lead',
 'me',
 'to',
 'believe',
 'that',
 'bromwell',
 "high's",
 'satire',
 'is',
 'much',
 'closer',
 'to',
 'reality',
 'than',
 'is',
 'teachers',
 'the',
 'scramble',
 'to',
 'survive',
 'financially',
 'the',
 'insightful',
 'students',
 'who',
 'can',
 'see',
 'right',
 'through',
 'their',
 'pathetic',
 "teachers'",
 'pomp',
 'the',
 'pettiness',
 'of',
 'the',
 'whole',
 'situation',
 'all',
 'remind',
 'me',
 'of',
 'the',
 'schools',
 'i',
 'knew',
 'and',
 'their',
 'students',
 'when',
 'i',
 'saw',
 'the',
 'episode',
 'in',
 'which',
 'a',
 'student',
 'repeatedly',
 'tried',
 'to',
 'burn',
 'down',
 'the',
 'school',
 'i',
 'immediately',
 'recalled',
 'at',
 'high',
 'a',
 'classic',
 'line',
 'insp

In [9]:
#去掉每条评论中的停用词
import nltk

In [10]:
from nltk.corpus import stopwords

In [11]:
english_stopwords = stopwords.words('english')

In [12]:
len(english_stopwords)

179

In [13]:
for i in range(len(x_train)):
    x_train[i] = [word for word in x_train[i] 
                  if not word in english_stopwords]

In [14]:
x_train[0]

['bromwell',
 'high',
 'cartoon',
 'comedy',
 'ran',
 'time',
 'programs',
 'school',
 'life',
 'teachers',
 '35',
 'years',
 'teaching',
 'profession',
 'lead',
 'believe',
 'bromwell',
 "high's",
 'satire',
 'much',
 'closer',
 'reality',
 'teachers',
 'scramble',
 'survive',
 'financially',
 'insightful',
 'students',
 'see',
 'right',
 'pathetic',
 "teachers'",
 'pomp',
 'pettiness',
 'whole',
 'situation',
 'remind',
 'schools',
 'knew',
 'students',
 'saw',
 'episode',
 'student',
 'repeatedly',
 'tried',
 'burn',
 'school',
 'immediately',
 'recalled',
 'high',
 'classic',
 'line',
 'inspector',
 "i'm",
 'sack',
 'one',
 'teachers',
 'student',
 'welcome',
 'bromwell',
 'high',
 'expect',
 'many',
 'adults',
 'age',
 'think',
 'bromwell',
 'high',
 'far',
 'fetched',
 'pity']

In [19]:

#tokenizer = Tokenizer(num_words=2000)  # 建立一个2000个单词的字典
#tokenizer.fit_on_texts(texts_filtered_stopwords)

In [37]:
#print(tokenizer.word_index)

{'bromwell': 1, 'high': 2, 'teachers': 3, 'school': 4, 'students': 5, 'student': 6, 'cartoon': 7, 'comedy': 8, 'ran': 9, 'time': 10, 'programs': 11, 'life': 12, '35': 13, 'years': 14, 'teaching': 15, 'profession': 16, 'lead': 17, 'believe': 18, "high's": 19, 'satire': 20, 'much': 21, 'closer': 22, 'reality': 23, 'scramble': 24, 'survive': 25, 'financially': 26, 'insightful': 27, 'see': 28, 'right': 29, 'pathetic': 30, "teachers'": 31, 'pomp': 32, 'pettiness': 33, 'whole': 34, 'situation': 35, 'remind': 36, 'schools': 37, 'knew': 38, 'saw': 39, 'episode': 40, 'repeatedly': 41, 'tried': 42, 'burn': 43, 'immediately': 44, 'recalled': 45, 'classic': 46, 'line': 47, 'inspector': 48, "i'm": 49, 'sack': 50, 'one': 51, 'welcome': 52, 'expect': 53, 'many': 54, 'adults': 55, 'age': 56, 'think': 57, 'far': 58, 'fetched': 59, 'pity': 60}


In [38]:
#x_train = tokenizer.texts_to_sequences(texts_filtered_stopwords)

In [34]:
"""
x_train_seq= {}
for index,word in enumerate(texts_filtered_stopwords):
    x_train_seq[index+1] = word
"""

In [15]:
import gensim
import warnings
warnings.filterwarnings("ignore")

E:\anaconda\envs\ml\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [16]:
#载入google_news预训练词向量
en_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [17]:
en_model['China'].shape[0]

300

In [18]:
en_model.vocab['student'].index  #每个词在google_news中都有一个index

930

In [22]:
en_model.index2word[0]

'</s>'

In [26]:
# 获得前100k个词的向量
num_words = 100000
embedding_matrix = np.zeros((num_words, 300))

for i in range(num_words):
    embedding_matrix[i,:] = en_model[en_model.index2word[i]]
embedding_matrix = embedding_matrix.astype('float32')

In [32]:
en_model.index2word[0]

'</s>'

In [27]:
embedding_matrix.shape

(100000, 300)

In [33]:
for text in x_train:
    for i, word in enumerate(text):
        try:
            # 将词转换为索引index
            text[i] = en_model.vocab[word].index
        except KeyError :
            # 如果词不在字典中，则输出0
            text[i] = 0
#train_tokens.append(cut_list)

In [34]:
x_train[22]

[3861,
 4364,
 112,
 11843,
 598,
 10893,
 314385,
 0,
 22101,
 1289,
 1311,
 154306,
 2738,
 11660,
 3691,
 16205,
 11536,
 1922,
 136460,
 1377413,
 582,
 1852,
 65,
 95774,
 359,
 41228,
 1279,
 14059,
 20931,
 537477,
 0,
 5024,
 15652,
 54,
 81,
 1067,
 11536,
 196,
 54,
 4313,
 441,
 1161,
 13217,
 3367,
 3213,
 1504,
 0,
 0,
 3407,
 11997,
 8751,
 3173,
 3781,
 112,
 155,
 459,
 213,
 56,
 1121,
 54,
 2485,
 941,
 598,
 16919,
 2532,
 2376,
 0,
 891,
 127,
 1021,
 1980,
 12675,
 1120,
 897,
 692,
 548,
 2118,
 0,
 18141,
 19527,
 17552,
 70159,
 0,
 1939,
 401,
 0,
 1885059,
 7293,
 251,
 1377413,
 8807,
 1922,
 621,
 32608,
 326,
 91,
 9136,
 370]

In [35]:
#让每条影片长度相同,假设固定为150，将超出的部分截掉,如果句子小于150，则用0在前面填充
x_train = sequence.pad_sequences(x_train,maxlen = 150)


In [36]:
x_train[1]

array([      0,       0,    5312,  359502,  126464,   13456,    2091,
         21264,    1200,    2242,     251,     838,     157,    5173,
           109,    4492,  100692,    2318,       0,       0,     158,
           588,    1852,   11138,     255,     294,   34073,   21264,
         14506,     757,     352,     455,     321,    1995,       0,
         21264,    3480,    1482,   23843,    2373,    3342,     274,
           408,   72120,     871,    9858,  342999,     482,    9919,
             0,  135037,    1513,   17911,   21264,    3346,    3304,
           143,     641,     521,  452539,       0,  176638,   95151,
          3116,   12825,     193,    5783,    1429,     124,   18498,
         18654,  368167,  407864,   23597,  136679,  321442,     311,
           233,    1852,   76476,    5320,   21264,  342999,     233,
          2770,    7421,    3592,      87,    2242,     821,    1809,
          2176,     496,     746,    2008,  452539,   21264,    1343,
          4123,     

In [37]:
# 超出前100k词的向量0代替
x_train[ x_train >=num_words ] = 0

In [41]:
x_train[4]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,  3772,     0,
           0,   692,   150,   373, 50177,  2579,   905,  4123,     0,
           0,     0, 95151,   107,  1121,  3903,  4261,  3934,  3424,
          75, 51906,   738,  3972,    75,   733,   529,   109,   835,
        1069,   422,

In [86]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding, LSTM, Bidirectional, Dropout, Flatten
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau

In [97]:
model=Sequential()

In [60]:
# model.add(Embedding(num_words,
#                     300,
#                     weights=[embedding_matrix],
#                     input_length=150,
#                     trainable=False))
# model.add(Bidirectional(LSTM(units=32, return_sequences=True)))
# model.add(LSTM(units=16, return_sequences=False))

In [91]:
# model.add(Dense(1, activation='sigmoid'))
# 我们使用adam以0.001的learning rate进行优化
# optimizer = Adam(lr=1e-3)

In [92]:
# model.compile(loss='binary_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])

In [98]:
#第一层为嵌入层，用之前的embedding_matrix进行初始化，并设置为不可训练
model.add(Embedding(num_words,
                    300,
                    weights=[embedding_matrix],
                    input_length=150,
                    trainable=False))
#第二层
model.add(Dense(128, input_dim=150 , activation='relu'))
model.add(Dropout(0.5))

#model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
#输出层
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])



In [99]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 150, 300)          30000000  
_________________________________________________________________
dense_18 (Dense)             (None, 150, 128)          38528     
_________________________________________________________________
dropout_5 (Dropout)          (None, 150, 128)          0         
_________________________________________________________________
dense_19 (Dense)             (None, 150, 64)           8256      
_________________________________________________________________
dropout_6 (Dropout)          (None, 150, 64)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9600)              0         
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 9601      
Total para

In [49]:
from sklearn.model_selection import train_test_split

In [91]:
X_train, X_test, y_train, y_test = train_test_split(x_train,
                                                    y_train,
                                                    test_size=0.1,
                                                    random_state=12)

In [102]:
model.fit(X_train, y_train,
          epochs=50,
          batch_size=32)
#score = model.evaluate(x_test, y_test, batch_size=128)

Epoch 1/50
22500/22500 [==============================] - 4s 191us/step - loss: 0.2093 - acc: 0.9176
Epoch 2/50
22500/22500 [==============================] - 4s 179us/step - loss: 0.2055 - acc: 0.9191
Epoch 3/50
22500/22500 [==============================] - 4s 181us/step - loss: 0.2075 - acc: 0.9178
Epoch 4/50
22500/22500 [==============================] - 4s 180us/step - loss: 0.2027 - acc: 0.9199
Epoch 5/50
22500/22500 [==============================] - 4s 182us/step - loss: 0.2004 - acc: 0.9212
Epoch 6/50
22500/22500 [==============================] - 4s 182us/step - loss: 0.1994 - acc: 0.9223
Epoch 7/50
22500/22500 [==============================] - 4s 182us/step - loss: 0.1972 - acc: 0.9219
Epoch 8/50
22500/22500 [==============================] - 4s 183us/step - loss: 0.1908 - acc: 0.9246
Epoch 9/50
22500/22500 [==============================] - 4s 180us/step - loss: 0.1899 - acc: 0.9260
Epoch 10/50
22500/22500 [==============================] - 4s 180us/step - loss: 0.1915 - a

In [106]:
result = model.evaluate(X_test, y_test)
print('Accuracy:{0:.2%}'.format(result[1]))

2500/2500 [==============================] - 0s 102us/step
Accuracy:86.60%


In [105]:
m = np.array(y_train) #转换为array


In [62]:
m.shape

(25000,)

In [67]:
xt = x_train.reshape(25000,150*300)
x_train

ValueError: cannot reshape array of size 3750000 into shape (25000,45000)

In [68]:
x_train.shape

(25000, 150)